In [6]:
!pip install numpy
!pip install pandas
!pip install sympy

Функция для задание начальных данных СЛАУ из примера 1

In [7]:
import numpy as np
from mpmath import mp
def create_matrix_1(n):
    A = np.full((n, n), 0.3)
    for i in range(n):
        A[i, i] = 1.0 + i * 0.0001
    b = [(i / 1) for i in range(1, len(A) + 1)]
    return A, b

Функция для задания начальных данных матрицы Гилберта из примера 2

In [8]:
def create_hilbert_matrix(n):
    i, j = np.indices((n, n))
    A = 1 / (i + j + 1)
    b = A.sum(axis=1)
    return A, b

Функция для задания начальных данных матрицы со случайными числами из примера 3

In [9]:
def create_random_matrix(n):
    A = np.random.randint(1, 1001, size=(n, n))
    b = np.random.randint(1, 1001, size=n).astype(float)  # Явное приведение к типу float
    return A, b

Метод Гаусса для последующего сравнения с методом цепных дробей

In [10]:
def gaussian_elimination_method(A, b):
    A = A.astype(float).copy()
    b = b.copy()

    size = len(b)
    for i in range(size):
        max_index = np.argmax(np.abs(A[i:, i])) + i
        A[[i, max_index]] = A[[max_index, i]]
        b[i], b[max_index] = b[max_index], b[i]

        pivot = A[i, i]
        A[i, :] /= pivot
        b[i] /= pivot

        coefficient = A[i + 1:, i].astype(float)
        A[i + 1:, :] -= np.outer(coefficient, A[i, :])
        b[i + 1:] -= coefficient * b[i]


    x = np.zeros(size)
    for i in range(size - 1, -1, -1):
        x[i] = b[i] - np.dot(A[i, i + 1:], x[i + 1:])

    return x

Метод простых итераций, который требуется для получения значений $x_i$.

Причем, мы сохраняем все приближения $x_i$ и по итогу получаем массив с 256 массивами, хранящими по 10 приближений $x_i$.

In [11]:
def simple_iteration_method(A, b, max_iterations=513, tolerance=1e-6):
    n = len(b)
    x = np.zeros(n)  # начальное приближение

    iterations = []

    for _ in range(max_iterations):
        x_new = np.zeros_like(x)

        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]

        iterations.append(x_new.copy())

        # Проверяем условие остановки
        if np.max(np.abs(x_new - x)) < tolerance:
            break

        x = x_new

    return iterations


Далее нам нужно получить из приближений $x_i$ значения $p_i$ по формуле $p_{i}^{(k)}$ = $x_{i}^{(k)}$ - $x_{i}^{(k-1)}$, где $k$ - номер итерации, а $i$ - номер корня

In [12]:
def get_p_list(x_iter_mat, ind):
    p_list = []
    x_ind_approx = []
    for i in range(len(x_iter_mat)):
        x_ind_approx.append(x_iter_mat[i][ind])

    p_list.append(x_ind_approx[0])
    for i in range(1, len(x_iter_mat)):
        p_list.append(x_ind_approx[i] - x_ind_approx[i - 1])
    return p_list

Затем принимая $\alpha_{1.k-1}^{i}$ за $p_{i}^{(k)}$ и $\alpha_{0.0}^{i}$ за $p_{i}^{(0)}$ вычисляем по рекурентным формулам Рутисхаузера значения $\alpha_{n.0}^{(i)}$, которые будут использоваться как коэффициенты цепной дроби

In [ ]:
def alpha(n, v, p):
    if n == 1:
        return p[v]
    elif n == 2:
        return alpha(1, v + 1, p) / alpha(1, v, p)
    elif n == 3:
        return -alpha(2, v + 1, p) + alpha(2, v, p)
    elif n % 2 == 0:
        return alpha(n - 2, v + 1, p) * alpha(n - 1, v + 1, p) / alpha(n - 1, v, p)  
    elif n % 2 == 1:
        return alpha(n - 2, v + 1, p) - alpha(n - 1, v + 1, p) + alpha(n - 1, v, p)
    else:
        pass # Ошибка

In [31]:
def alpha(n, v, p, memo={}, epsilon=1e-10):
    if (n, v) in memo:
        return memo[(n, v)]

    result = 0.0
    
    if n == 1:
        result = p[v]
    elif n == 2:
        denominator = alpha(1, v, p, memo)
        if abs(denominator) < epsilon:
            result = 0.0
        else:
            result = alpha(1, v + 1, p, memo) / denominator
    elif n == 3:
        denominator = alpha(2, v, p, memo)
        if abs(denominator) < epsilon:
            result = alpha(2, v + 1, p, memo)
        else:
            result = (-alpha(2, v + 1, p, memo) + alpha(2, v, p, memo)) / denominator
    elif n % 2 == 0 and n > 2:
        denominator = alpha(n - 1, v, p, memo)
        if abs(denominator) < epsilon:
            result = 0.0
        else:
            result = (alpha(n - 2, v + 1, p, memo) * alpha(n - 1, v + 1, p, memo)) / denominator
    elif n % 2 == 1 and n > 3:
        denominator = alpha(n - 1, v, p, memo)
        if abs(denominator) < epsilon:
            result = alpha(n - 2, v + 1, p, memo) - alpha(n - 1, v + 1, p, memo)
        else:
            result = (alpha(n - 2, v + 1, p, memo) - alpha(n - 1, v + 1, p, memo) + alpha(n - 1, v, p, memo)) / denominator

    memo[(n, v)] = result
    return result

In [92]:
def get_alpha_list(p_lists):
    mp.dps = 20  # Устанавливаем точность

    alpha_lists = []
    for i in range(256):
        n = len(p_lists[i])
        matrix = np.zeros((n, n), dtype=object)  # Используем object dtype для хранения mp.mpf

        # Заполняем первую строку матрицы
        matrix[0, :] = [mp.mpf(str(x)) for x in p_lists[i]]

        # Рекуррентно заполняем остальные строки матрицы
        for k in range(2, n):
            for v in range(1, n - k + 1):
                if k == 2:
                    denominator = mp.mpf(matrix[k - 2, v - 1])
                    matrix[k - 1, v - 1] = mp.mpf(matrix[k - 2, v]) / denominator
                elif k == 3:
                    matrix[k - 1, v - 1] = (mp.mpf(matrix[k - 2, v - 1]) - mp.mpf(matrix[k - 2, v]))
                elif k % 2 == 0:
                    denominator = mp.mpf(matrix[k - 2, v - 1])
                    matrix[k - 1, v - 1] = (mp.mpf(matrix[k - 3, v]) * mp.mpf(matrix[k - 2, v])) / denominator
                else:
                    matrix[k - 1, v - 1] = (mp.mpf(matrix[k - 3, v]) - mp.mpf(matrix[k - 2, v]) + mp.mpf(matrix[k - 2, v - 1]))

        alpha_lists.append(matrix[:, 0])
    return alpha_lists

Далее вычисляем саму цепную дробь, ее значение и будет корнем СЛАУ

In [84]:
def calculate_chain_fraction(a, index=0): # Вычисляем значение цепной дроби
    # Базовый случай: если достигнут конец массива
    if (index == len(a) - 1 ) or a[index] == 0:
        return a[index]

    # Рекурсивный случай: вычисляем цепную дробь для текущего элемента
    next_term = calculate_chain_fraction(a, index + 1)
    
    if index % 2 == 0:
        return a[index] / (1 - next_term)
    else:
        return a[index] / (1 + next_term)

Функция решения СЛАУ методом суммирования расходящихся рядов

In [85]:
def chain_fraction_method(A, b, max_iter=10, eps=1e-100):
    x_iter_history = simple_iteration_method(A, b, max_iterations=max_iter)
    p_lists = np.array([get_p_list(x_iter_history, i) for i in range(len(A))])
    alpha_lists = get_alpha_list(p_lists)
    result_list = [calculate_chain_fraction(alpha_lists[i]) for i in range(len(A))]
    return result_list

Функция для анализ решения системы

In [86]:
import pandas as pd
import time
def analise(A, b, max_iter = 10):
    start_chain = time.time()
    result = chain_fraction_method(A, b, max_iter)
    end_chain = time.time()
    time_chain = end_chain - start_chain

    start_gaus = time.time()
    gaus = gaussian_elimination_method(A, b)
    end_gaus = time.time()
    time_gaus = end_gaus - start_gaus

    print(f"Время работа метода цепных дробей: {round(time_chain, 5)}")
    print(f"Время работа метода Гаусса: {round(time_gaus, 5)}")
    
    df = pd.DataFrame(columns=['Номер корня','Значение по методу цепных дробей',
                            'Значение по методу Гаусса', 
                            'Разница между корнями'])
    for i in range(len(A)):
        tmp = [i + 1, result[i], gaus[i], abs(result[i] - gaus[i])]
        df.loc[len(df)] = tmp
    df['Номер корня'] = df['Номер корня'].astype(int)
    df.set_index('Номер корня', inplace=True)
    return df

Анализ СЛАУ (1)

In [95]:
A_1, b_1 = create_matrix_1(256)
analise(A_1, b_1, 20).head(256)

Время работа метода цепных дробей: 0.31371
Время работа метода Гаусса: 0.01504


,Значение по методу цепных дробей,Значение по методу Гауса,Разница между корнями
Номер корня,,,
1,-179.37039936030590468,-179.370417,0.000017727311322342622149
2,-177.91642373475180084,-177.916429,5.2915760465515378902e-6
3,-176.46282569774363037,-176.462856,0.000030573796062069663951
4,-175.00970566416019901,-175.009699,7.0188194366395482288e-6
5,-173.55695437881562456,-173.556956,1.5911047562082707846e-6
...,...,...,...
252,172.99781437379638866,172.997804,9.8797019926868175771e-6
253,174.35293759602387459,174.352879,0.000058336967296017351245
254,175.70759379755651669,175.707580,0.000013432648505225285376


Анализ СЛАУ (2)

In [97]:
A_2, b_2 = create_hilbert_matrix(256)
analise(A_2, b_2, 100).head(256)

Время работа метода цепных дробей: 7.05802
Время работа метода Гаусса: 0.01181


,Значение по методу цепных дробей,Значение по методу Гауса,Разница между корнями
Номер корня,,,
1,0.84330909154105697566,1.000000,0.15669128533827698747
2,1.127146098261871561,0.999929,0.12721742881218061771
3,1.1215040810760656253,1.003139,0.11836492390349644064
4,1.0819828228996634535,0.944163,0.1378202942830903267
5,1.0403623229429877801,1.486719,0.44635670892433601247
...,...,...,...
252,1.0092096149342382989,-14.305769,15.31497905204071527
253,1.0126234829538318301,-97.995679,99.008302134198278485
254,1.0131773817839588353,-131.054743,132.06792072923566378


In [99]:
A_3, b_3 = create_random_matrix(256)
analise(A_3, b_3, 12).head(256)

Время работа метода цепных дробей: 0.11947
Время работа метода Гаусса: 0.01127


,Значение по методу цепных дробей,Значение по методу Гауса,Разница между корнями
Номер корня,,,
1,10.034029143033978512,0.457188,9.5768415502718103744
2,0.66103988100607065072,-0.307220,0.96825963636537311064
3,1.1725262749779273247,0.536305,0.63622118820238898253
4,10.256676148178717604,-0.129026,10.385702036824755749
5,-1.4409964109006186277,1.210418,2.6514144674099640468
...,...,...,...
252,-3.2298286888618913201,-0.001163,3.2286655639557447581
253,-1.306345730814340136,-0.431431,0.87491439442795904361
254,11.524388897198766431,0.024625,11.499763623885740767
